In [2]:
#Loading training and test datasets

#importing libraries and dependancies
import numpy as np
import gzip
import h5py
import os


#File paths for train and test datasets in data dir
TRAIN_PATH = 'cullpdb+profile_6133_filtered.npy'
TEST_PATH = 'cb513+profile_split1.npy'
CASP10_PATH = 'casp10.h5'
CASP11_PATH = 'casp11.h5'


#load filtered cullpdb training data, all_data - proportion of data to load in, default is 1 which is whole dataset
def load_cul6133_filted(all_data=1):

    print("Loading training dataset (Cullpdb_filtered)...\n")

    #load dataset
    data = np.load(TRAIN_PATH)
    #reshape dataset
    data = np.reshape(data, (-1, 700, 57))
    #sequence feature
    datahot = data[:, :, 0:21]
    #profile features
    datapssm = data[:, :, 35:56]
    #secondary struture labels
    labels = data[:, :, 22:30]
    #np.random.seed(2018)

    # shuffle data
    num_seqs, seqlen, feature_dim = np.shape(data)
    num_classes = labels.shape[2]
    seq_index = np.arange(0, num_seqs)#
    np.random.shuffle(seq_index)

    #calculate the indexes for each dimension based on all_data input parameter
    data_index = int(5278 * all_data)
    val_data_index =  int(256 * all_data)
    val_data_upper = data_index + val_data_index

    #get training data
    trainhot = datahot[seq_index[:data_index]]
    trainlabel = labels[seq_index[:data_index]]
    trainpssm = datapssm[seq_index[:data_index]]

    #get validation data
    vallabel = labels[seq_index[data_index:val_data_upper]] #8
    valpssm = datapssm[seq_index[data_index:val_data_upper]] # 21
    valhot = datahot[seq_index[data_index:val_data_upper]] #21


    train_hot = np.ones((trainhot.shape[0], trainhot.shape[1]))
    for i in range(trainhot.shape[0]):
        for j in range(trainhot.shape[1]):
            if np.sum(trainhot[i,j,:]) != 0:
                train_hot[i,j] = np.argmax(trainhot[i,j,:])


    val_hot = np.ones((valhot.shape[0], valhot.shape[1]))
    for i in range(valhot.shape[0]):
        for j in range(valhot.shape[1]):
            if np.sum(valhot[i,j,:]) != 0:
                val_hot[i,j] = np.argmax(valhot[i,j,:])

    return train_hot,trainpssm,trainlabel, val_hot,valpssm,vallabel



In [3]:
train_hot,trainpssm,trainlabel, val_hot,valpssm,vallabel = load_cul6133_filted()

Loading training dataset (Cullpdb_filtered)...



In [ ]:
def get_data_labels(D):
    X = D[:, :, 0:amino_acid_residues]
    Y = D[:, :, amino_acid_residues:amino_acid_residues + num_classes]
    return X, Y

In [ ]:
D_train, D_test, D_val = split_with_shuffle(dataset, 100)

X_train, Y_train = get_data_labels(D_train)

In [ ]:
def Q8_accuracy(real, pred):
    total = real.shape[0] * real.shape[1]
    correct = 0
    for i in range(real.shape[0]):  # per element in the batch
        for j in range(real.shape[1]): # per aminoacid residue
            if np.sum(real[i, j, :]) == 0:  #  real[i, j, dataset.num_classes - 1] > 0 # if it is padding
                total = total - 1
            else:
                if real[i, j, np.argmax(pred[i, j, :])] > 0:
                    correct = correct + 1

    return correct / total

net.load_weights("Whole_CullPDB-best.hdf5")

predictions = net.predict(X_test)

print("\n\nQ8 accuracy: " + str(model.Q8_accuracy(Y_test, predictions)) + "\n\n")

